In [ ]:
import pandas as pd
import numpy as np
import csv
import os
from datetime import datetime

In [ ]:
!pip install -r requirements.txt

In [ ]:
date_cols = ['Время получения', 'Время прочтения', 'Время отправки ответа']
email_dataset = pd.read_csv('samples/modified_email_dataset.csv', sep=';', encoding='utf-8', parse_dates=date_cols)

In [ ]:
# fix datetime format of columns with adding seconds where values is not nan
# NOT IMPLEMENT ANY MORE!
for col in date_cols:
    # get indexes of not nan values
    not_nan_indexes = email_dataset[col].notna()
    # add seconds to values if length of value is 16
    email_dataset.loc[not_nan_indexes, col] = email_dataset.loc[not_nan_indexes, col].apply(
        lambda x: x + ':00' if len(x) == 16 else x)


In [ ]:
#  parse columns to datetime
for col in date_cols:
    email_dataset[col] = pd.to_datetime(email_dataset[col], format='%Y-%m-%d %H:%M:%S')

# ML models

In [ ]:
# install required packages
!pip install torch transformers sentence-transformers

In [ ]:
# delete conflict versions of numpy
!pip install --upgrade transformers

## Emotion model

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from enum import Enum

emotion_tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rurewiews')
emotion_model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rurewiews', return_dict=True)

class EmotionDetection():
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    class Emotion(Enum):
        NEUTRAL = 0
        POSITIVE = 1
        NEGATIVE = 2

    def predict(self, text) -> Emotion:
        inputs = self.tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
        outputs = self.model(**inputs)
        predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
        predicted = torch.argmax(predicted, dim=1).numpy()
        return EmotionDetection.Emotion(predicted[0])

# 0: NEUTRAL
# 1: POSITIVE
# 2: NEGATIVE
model_emotions = EmotionDetection(emotion_tokenizer, emotion_model)
# example:
# e
# emotion_model.predict("Да отвали ты от меня")

In [ ]:
model_emotions.predict("Да отвали ты от меня")

## Unanswered questions model

In [ ]:
from sentence_transformers import SentenceTransformer, util
import re
counter_model = SentenceTransformer('sentence-transformers/use-cmlm-multilingual')


class UnansweredCounterBase():
    def __init__(self, model):
        self.model = model

    def extract_questions(self, text):
        return [t.strip() for t in re.findall(r'[^.!?]*\?', text)]

    def extract_sentences(self, text):
        return [t.strip() for t in re.findall(r'[^.!?]+[.!?]', text)]

    def prepare_context(self, context):
        return self.extract_questions(context)

    def prepare_answer(self, answer):
        return self.extract_sentences(answer)

context = """Добрый день, Илья!

Надеюсь, этот день проходит у вас продуктивно. Я пишу, чтобы уточнить несколько важных моментов по нашему текущему проекту:

Можете ли вы обновить меня относительно текущего статуса проекта? Какие основные этапы уже завершены?

Как обстоят дела в команде? Есть ли какие-то сложности или препятствия, о которых нам следует знать?

Соблюдаются ли запланированные сроки? Есть ли риск задержек в достижении ключевых вех проекта?

Буду признателен за оперативный ответ, так как это поможет нам в планировании следующих шагов.

С уважением,
Егор
"""
answer = """Добрый день, Егор!

Спасибо за ваше обращение. Я рад сообщить вам о последних достижениях нашего проекта:

На данный момент мы успешно завершили первые два этапа проекта, включая исследовательскую фазу и начальную разработку. В настоящее время мы активно работаем над следующим этапом, который ориентирован на разработку основных функциональностей.

Что касается команды, все идет хорошо. Несмотря на некоторые начальные трудности с координацией задач, на данный момент все проблемы решены, и команда работает слаженно.

Если у вас возникнут дополнительные вопросы, не стесняйтесь обращаться.

С уважением,
Илья"""

class UnansweredCounter():
    def __init__(self, context: str, answer: str, model):
        self.unanswered_counter = model
        self.context = context
        self.answer = answer

    def count(self) -> int:
        context = self.unanswered_counter.prepare_context(self.context)
        answer = self.unanswered_counter.prepare_answer(self.answer)
        num_questions = len(context)
        count = 0
        for question in context:
            embedding_1 = self.unanswered_counter.model.encode(question, convert_to_tensor=True)
            for sentence in answer:
                embedding_2 = self.unanswered_counter.model.encode(sentence, convert_to_tensor=True)
                score = util.pytorch_cos_sim(embedding_1, embedding_2)
                if score >= 0.27:
                    count += 1
                    break
        return num_questions - count



# unanswered_counter = UnansweredCounterBase(counter_model)
# context = unanswered_counter.prepare_context(context)
# answer = unanswered_counter.prepare_answer(answer)

# count = 0
# for question in context:
#     embedding_1 = unanswered_counter.model.encode(question, convert_to_tensor=True)
#     for sentence in answer:
#         embedding_2 = unanswered_counter.model.encode(sentence, convert_to_tensor=True)
#         score = util.pytorch_cos_sim(embedding_1, embedding_2)
#         if score >= 0.27:
#           print("----------------------------------------------------------------------------")
#           print(f"Context: {question}")
#           print(f"Answer: {sentence}")
#           print(f"Similarity: {score}")
#           print("----------------------------------------------------------------------------")
#           count += 1
#           break
# print(count)
base_counter_model = UnansweredCounterBase(counter_model)
unAnsweredCounter = UnansweredCounter(context, answer, base_counter_model)

In [ ]:
unAnsweredCounter.count()

## Toxcicity model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import nn

class ToxicLanguageIndetifier(nn.Module):
    def __init__(self, model_name = "s-nlp/russian_toxicity_classifier"):
        super().__init__()
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def toxic_probality(self, text):
        text_toxicity = self.tokenizer(text, return_tensors='pt')
        preds = self.model(**text_toxicity).logits
        return ['neutral', 'toxic'][preds.softmax(dim=-1).argmax(dim=-1)]

toxic_model = ToxicLanguageIndetifier()

In [ ]:
toxic_model.toxic_probality("Сбер - контора пидорасов")

## Priority model

In [ ]:
class PrioriotyModel(nn.Module):
    def __init__(self, model_name="fathyshalaby/emailclassifier", classes=['высокая срочность', 'средняя срочность', 'не срочно']):
        super().__init__()
        self.model_name = model_name
        self.classes = classes
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.classifier = nn.Linear(in_features=768, out_features=len(classes), bias=True)

    def get_priority(self, text: str):

        tokenized_text = self.tokenizer(text, return_tensors='pt')
        preds = self.model(**tokenized_text).logits
        return self.classes[preds.softmax(dim=-1).argmax(dim=-1)]

priority_model = PrioriotyModel()

In [ ]:
priority_model.get_priority("Добрый день, возник вопрос по системе учета переработок. Какие шаги нужно предпринять, чтобы получить информацию по своему рабочему времени?")

# Define metrics functions

In [ ]:
import random

def random_salary():
    salary_range = list(range(20000, 100001, 5000))
    weights = [i for i in range(1, len(salary_range) + 1)]
    weights.reverse()
    print(weights)
    return random.choices(salary_range, weights=weights)[0]

# 8
def number_of_answered_emails(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    # return count of answered emails
    return df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время отправки ответа'].notna())].shape[0]

# 1
def number_of_sent_emails(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    # return count of sent emails by user_email between start_date and end_date
    sender_count =  df[(df['Почтовый адрес отправителя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)].shape[0]
    answer_count = number_of_answered_emails(user_email, start_date, end_date, df)
    return sender_count + answer_count

# 2
def number_of_received_emails(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    # return count of received emails by user_email between start_date and end_date
    return df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)].shape[0]

# 3
def mean_number_of_recipients_in_one_email_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> float:
    # generate random number from 1 to 5
    return np.random.randint(1, 5)

# 6
def number_of_emails_read_after_x_minutes(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame, minutes: int = 1) -> int:
    # return count of emails read after x minutes after receiving
    return df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время прочтения'] - df['Время получения'] >= pd.Timedelta(minutes=minutes))].shape[0]

# 7
def mean_number_of_days_between_receiving_emails_and_read(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    # return mean number of days between receiving emails and read
    data = df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время прочтения'].notna())]
    return data['Время прочтения'].sub(data['Время получения']).mean().days

# 9
def mean_length_of_user_emails(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> float:
    # return mean length of user emails text. Both sent and answered
    mean_length_sent = df[(df['Почтовый адрес отправителя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)]['Текст письма'].str.len().mean()
    mean_length_answered = df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время отправки ответа'].notna())]['Текст ответа'].str.len().mean()
    return (mean_length_sent + mean_length_answered) / 2

# 10
def number_of_sent_emails_outside_of_working_hours(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    working_hours = (10, 18)
    # return count of sent emails outside of working hours
    sent_emails = df[(df['Почтовый адрес отправителя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)]
    answered_emails = df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время отправки ответа'].notna())]
    # apply working hours filter
    sent_emails = sent_emails[(sent_emails['Время получения'].dt.hour < working_hours[0]) | (sent_emails['Время получения'].dt.hour > working_hours[1])]
    answered_emails = answered_emails[(answered_emails['Время отправки ответа'].dt.hour < working_hours[0]) | (answered_emails['Время отправки ответа'].dt.hour > working_hours[1])]
    return sent_emails.shape[0] + answered_emails.shape[0]


# 11
def received_and_sent_emails_proportion_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> float:
    return number_of_received_emails(user_email, start_date, end_date, df) / number_of_sent_emails(user_email, start_date, end_date, df)

# 13
def number_of_not_answered_questions_in_email(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> float:
    # return count of not answered questions in email using ML
    total_emails = number_of_answered_emails(user_email, start_date, end_date, df)
    count = 0
    for index, row in df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время отправки ответа'].notna())].iterrows():
        unanswered_counter = UnansweredCounter(row['Текст письма'], row['Текст ответа'], base_counter_model)
        count += unanswered_counter.count()
    return count / total_emails

# 14
def salary_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    return random_salary()

# 15
def mean_working_hours_per_day_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> float:
    # return random number from 6 to 12
    return np.random.randint(6, 12)

def reply_delay_for_prioritised_emails(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> dict:
    # return dict with count of emails with different priority and mean delay for each priority
    # return random dict
    return {
        'высокая срочность': np.random.randint(5, 180),
        'средняя срочность': np.random.randint(20, 300),
        'не срочно': np.random.randint(60, 480)
    }

# TODO define the returning type
def emotions_in_sent_emails_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> str:
    # return main emotion for each sent and answered email
    total_eamils = number_of_sent_emails(user_email, start_date, end_date, df)
    emotion_temperature = 0
    for index, row in df[(df['Почтовый адрес отправителя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)].iterrows():
        value = model_emotions.predict(row['Текст письма']).value
        if value == 2:
            value = -1
        emotion_temperature += value
    for index, row in df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время отправки ответа'].notna())].iterrows():
        value = model_emotions.predict(row['Текст ответа']).value
        if value == 2:
            value = -1
        emotion_temperature += value
    mean_temperature = emotion_temperature / total_eamils
    # if mean_temperature is close to 0 then return neutral. If mean_temperature is close to 1 then return positive. If mean_temperature is close to -1 then return negative
    if mean_temperature > 0.3:
        return 'POSITIVE'
    elif mean_temperature < -0.3:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

# TODO define the returning type
def emotions_in_received_emails_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    # return main emotion for each received email
    total_emails = number_of_received_emails(user_email, start_date, end_date, df)
    emotion_temperature = 0
    for index, row in df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)].iterrows():
        value = model_emotions.predict(row['Текст письма']).value
        if value == 2:
            value = -1
        emotion_temperature += value
    mean_temperature = emotion_temperature / total_emails
    # if mean_temperature is close to 0 then return neutral. If mean_temperature is close to 1 then return positive. If mean_temperature is close to -1 then return negative
    if mean_temperature > 0.3:
        return 'POSITIVE'
    elif mean_temperature < -0.3:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'

def toxcity_in_sent_emails_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> float:
    # return mean toxicity of each sent and answered email
    total_emails = number_of_sent_emails(user_email, start_date, end_date, df)
    toxicity_counter = 0
    for index, row in df[(df['Почтовый адрес отправителя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)].iterrows():
        toxicity_counter += toxic_model.toxic_probality(row['Текст письма']) == 'toxic'
    for index, row in df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время отправки ответа'].notna())].iterrows():
        toxicity_counter += toxic_model.toxic_probality(row['Текст ответа']) == 'toxic'
    toxic_level = toxicity_counter / total_emails
    if toxic_level > 0.3:
        return 'HIGH'
    elif toxic_level < 0.1:
        return 'LOW'
    else:
        return 'MEDIUM'

def toxcity_in_received_emails_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> float:
    # return mean toxicity of each received email
    total_emails = number_of_received_emails(user_email, start_date, end_date, df)
    toxicity_counter = 0
    for index, row in df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)].iterrows():
        toxicity_counter += toxic_model.toxic_probality(row['Текст письма']) == 'toxic'
    toxic_level = toxicity_counter / total_emails
    if toxic_level > 0.3:
        return 'HIGH'
    elif toxic_level < 0.1:
        return 'LOW'
    else:
        return 'MEDIUM'

# TODO define the returning type
def mean_answering_time_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> float:
    # return mean answering time for user
    data = df[(df['Почтовый адрес получателя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date) & (df['Время отправки ответа'].notna())]
    return data['Время отправки ответа'].sub(data['Время получения']).mean().seconds / 60
def number_of_passed_corporative_tests_or_courses_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    # return count of passed corporative tests or courses for user. Return random number from 0 to 3
    return np.random.randint(0, 3)

def number_of_unique_recipients_of_emails_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    # return count of unique recipients of emails for user
    return df[(df['Почтовый адрес отправителя'] == user_email) & (df['Время получения'] >= start_date) & (df['Время получения'] <= end_date)]['Почтовый адрес получателя'].nunique()

def number_of_unique_departments_in_emails_for_user(user_email: str, start_date: datetime, end_date: datetime, df: pd.DataFrame) -> int:
    # return count of unique departments in emails for user. Return random number from 1 to 5
    return np.random.randint(1, 5)

In [ ]:
mean_answering_time_for_user('49@company.com', datetime(2020, 1, 1), datetime(2030, 1, 1), email_dataset)


In [ ]:
email_dataset[email_dataset['Почтовый адрес получателя'] == '49@company.com' ]

In [ ]:
# fix dataset on sent emails
# if there is no text of reply email, then set answer time end theme of email to nan
email_dataset.loc[(email_dataset['Текст ответа'].isna()), 'Время отправки ответа'] = pd.NaT
email_dataset.loc[(email_dataset['Текст ответа'].isna()), 'Тема ответа'] = np.nan

In [ ]:
email_dataset.to_csv('export/final_email_dataset.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
# get lowest date and highest date from dataset
lowest_date = email_dataset['Время получения'].min()
highest_date = email_dataset['Время получения'].max()
print(f'low: {lowest_date}; high: {highest_date}')

In [ ]:
from datetime import timedelta, datetime
def get_same_weekday_date(input_date: datetime) -> datetime:
    target_weekday = input_date.weekday()
    start_date = datetime(2023, 12, 4)
    end_date = datetime(2023, 12, 10)

    # Find the first date with the same weekday as the input date
    current_date = start_date
    while current_date.weekday() != target_weekday:
        current_date += timedelta(days=1)

    # Check if the found date is within the desired range
    if current_date < start_date:
        current_date += timedelta(days=7)
    elif current_date > end_date:
        current_date -= timedelta(days=7)

    # Save the time of the input_date to the output datetime
    current_date = current_date.replace(hour=input_date.hour, minute=input_date.minute, second=input_date.second, microsecond=input_date.microsecond)

    return current_date


In [ ]:
# replace all dates in dataset with same weekday dates from function above. Only non nan values
for col in date_cols:
    not_nan_indexes = email_dataset[col].notna()
    email_dataset.loc[not_nan_indexes, col] = email_dataset.loc[not_nan_indexes, col].apply(get_same_weekday_date)


In [ ]:
email_dataset.to_csv('export/modified_email_dataset.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
import random

def random_salary():
    salary_range = list(range(20000, 100001, 5000))
    weights = [i for i in range(1, len(salary_range) + 1)]
    weights.reverse()
    print(weights)
    return random.choices(salary_range, weights=weights)[0]




In [ ]:
salary = random_salary()
print(salary)

# Metrics dataset generator

In [ ]:
import tqdm
# generate metrics for each user
# get all unique users from dataset from column 'Почтовый адрес получателя' and 'Почтовый адрес отправителя'
unique_users = set(email_dataset['Почтовый адрес получателя'].unique().tolist() + email_dataset['Почтовый адрес отправителя'].unique().tolist())

# set delay threshold in minutes
delay_trshld = 60
# generate metrics for each user
metrics_dataset = pd.DataFrame(columns=['Почтовый адрес',
                                        'Колличество ответов',
                                        'Количество отправленных писем',
                                        'Количество полученных писем',
                                        'Среднее количество получателей в письме',
                                        f'Количество писем, прочитанных через {delay_trshld} минут после получения',
                                        'Среднее количество дней между получением и прочтением письма',
                                        'Количество писем, отправленных в нерабочее время',
                                        'Соотношение полученных и отправленных писем',
                                        'Количество неотвеченных вопросов в письмах',
                                        'Средняя длина письма',
                                        'Среднее время ответа на письмо',
                                        'Количество пройденных корпоративных тестов и курсов',
                                        'Количество уникальных получателей писем',
                                        'Количество уникальных отделов в письмах',
                                        'Средняя токсичность отправленных писем',
                                        'Средняя токсичность полученных писем',
                                        'Средняя эмоциональность отправленных писем',
                                        'Средняя эмоциональность полученных писем',
                                        'Зарплата'])
for user in tqdm.tqdm(unique_users):
    data = {
        'Почтовый адрес': user,
        'Колличество ответов': number_of_answered_emails(user, lowest_date, highest_date, email_dataset),
        'Количество отправленных писем': number_of_sent_emails(user, lowest_date, highest_date, email_dataset),
        'Количество полученных писем': number_of_received_emails(user, lowest_date, highest_date, email_dataset),
        'Среднее количество получателей в письме': mean_number_of_recipients_in_one_email_for_user(user, lowest_date, highest_date, email_dataset),
        f'Количество писем, прочитанных через {delay_trshld} минут после получения': number_of_emails_read_after_x_minutes(user, lowest_date, highest_date, email_dataset, delay_trshld),
        'Среднее количество дней между получением и прочтением письма': mean_number_of_days_between_receiving_emails_and_read(user, lowest_date, highest_date, email_dataset),
        'Количество писем, отправленных в нерабочее время': number_of_sent_emails_outside_of_working_hours(user, lowest_date, highest_date, email_dataset),
        'Соотношение полученных и отправленных писем': received_and_sent_emails_proportion_for_user(user, lowest_date, highest_date, email_dataset),
        'Количество неотвеченных вопросов в письмах': number_of_not_answered_questions_in_email(user, lowest_date, highest_date, email_dataset),
        'Средняя длина письма': mean_length_of_user_emails(user, lowest_date, highest_date, email_dataset),
        'Среднее время ответа на письмо': mean_answering_time_for_user(user, lowest_date, highest_date, email_dataset),
        'Количество пройденных корпоративных тестов и курсов': number_of_passed_corporative_tests_or_courses_for_user(user, lowest_date, highest_date, email_dataset),
        'Количество уникальных получателей писем': number_of_unique_recipients_of_emails_for_user(user, lowest_date, highest_date, email_dataset),
        'Количество уникальных отделов в письмах': number_of_unique_departments_in_emails_for_user(user, lowest_date, highest_date, email_dataset),
        'Средняя токсичность отправленных писем': toxcity_in_sent_emails_for_user(user, lowest_date, highest_date, email_dataset),
        'Средняя токсичность полученных писем': toxcity_in_received_emails_for_user(user, lowest_date, highest_date, email_dataset),
        'Средняя эмоциональность отправленных писем': emotions_in_sent_emails_for_user(user, lowest_date, highest_date, email_dataset),
        'Средняя эмоциональность полученных писем': emotions_in_received_emails_for_user(user, lowest_date, highest_date, email_dataset),
        'Зарплата': salary_for_user(user, lowest_date, highest_date, email_dataset)
    }
    metrics_dataset = pd.concat([metrics_dataset, pd.DataFrame([data])], ignore_index=True)

metrics_dataset.to_csv('export/metrics_dataset.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
# replace fixed columns 'Среднее количество дней между получением и прочтением письма' and 'Среднее время ответа на письмо' with new values
metrics_dataset['Среднее количество дней между получением и прочтением письма'] = metrics_dataset['Почтовый адрес'].apply(lambda x: mean_number_of_days_between_receiving_emails_and_read(x, lowest_date, highest_date, email_dataset))
metrics_dataset['Среднее время ответа на письмо'] = metrics_dataset['Почтовый адрес'].apply(lambda x: mean_answering_time_for_user(x, lowest_date, highest_date, email_dataset))

In [ ]:
# add new columns for reply_delay_for_prioritised_emails. Each column is a key of dict
for user in tqdm.tqdm(unique_users):
    data = reply_delay_for_prioritised_emails(user, lowest_date, highest_date, email_dataset)
    # add new columns for each key in dict
    for key in data.keys():
        metrics_dataset.loc[metrics_dataset['Почтовый адрес'] == user, key] = data[key]

In [ ]:
metrics_dataset.to_csv('export/metrics_dataset.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
# fix metrics dataset of column 'Среднее количество дней между получением и прочтением письма' with multiplying by -1
metrics_dataset['Среднее количество дней между получением и прочтением письма'] = metrics_dataset['Среднее количество дней между получением и прочтением письма'].apply(lambda x: x * -1)

In [ ]:
metrics_dataset.to_csv('export/metrics_dataset.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
from datetime import datetime, timedelta

def get_next_week_dates(date_str):
    # Convert the input string to a datetime object
    given_date = datetime.strptime(date_str, "%Y-%m-%d")

    # Find out the day of the week (0=Monday, 6=Sunday)
    day_of_week = given_date.weekday()

    # Calculate the start of the next week (next Monday)
    # If the given day is Monday, we need to add 7 days to get to the next Monday
    days_till_next_monday = 7 - day_of_week
    start_of_next_week = given_date + timedelta(days=days_till_next_monday)

    # End of the next week (next Sunday) is 6 days after the start of the week
    end_of_next_week = start_of_next_week + timedelta(days=6)

    # Return the dates as strings
    return start_of_next_week, end_of_next_week

In [ ]:
metrics_dataset.columns